# IMPORT

In [1]:
from __future__ import annotations
from qiskit import QuantumCircuit
from transpiler.qasm2_reader import load_qasm, from_qasm2_to_braket
from bqskit.ext import bqskit_to_qiskit,qiskit_to_bqskit
from qiskit_aer import AerSimulator
from qiskit import transpile
import pennylane as qml
import numpy as np
from braket.devices import LocalSimulator
from braket.circuits import Circuit as braket_Circuit
from transpiler.transpile import transpile_circuit as bqskit_transpiler
from transpiler.transpile import get_noisy_counts
from qiskit.circuit.random import random_circuit

In [17]:
# Create a 4-qubit quantum circuit
# Generate a random 4-qubit circuit with a depth of 5
qc = random_circuit(4,max_operands=2, depth=4, measure=False)

    
qc_qiskit = qc
qc_bqskit=qiskit_to_bqskit(qc)
print(qc)

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

      ┌────┐     ┌───┐     ┌─────────────┐              »
q_0: ─┤ √X ├─────┤ Z ├─────┤ Rz(0.62416) ├──────────────»
     ┌┴────┴┐┌───┴───┴────┐└───┬─────┬───┘              »
q_1: ┤0     ├┤ Ry(3.4708) ├────┤ Sdg ├──────────────────»
     │  Ecr │└──┬─────┬───┘    └──┬──┘    ┌────────────┐»
q_2: ┤1     ├───┤ Tdg ├───────────┼───────┤ Rx(3.8194) ├»
     └┬────┬┘   ├─────┤           │       └────────────┘»
q_3: ─┤ √X ├────┤ Sdg ├───────────■─────────────────────»
      └────┘    └─────┘                                 »
«     ┌────────────────────────┐     
«q_0: ┤0                       ├─────
«     │                        │┌───┐
«q_1: ┤  (XX+YY)(6.1596,2.446) ├┤ X ├
«     │                        │└─┬─┘
«q_2: ┤1                       ├──┼──
«     └────────────────────────┘  │  
«q_3: ────────────────────────────■──
«                                    
Not transpiled result: [0.17558095 0.18633726 0.00484553 0.00514237 0.05694874 0.06770506
 0.00157162 0.00186846 0.00484553 0.00514237 0

In [30]:
# Step 3: Transpile the circuit to MS gates
qc_qiskit_transpiled = bqskit_transpiler(qc_qiskit,device="Aria",optimization_level=2,synthesis_epsilon=1e-4)
print(qc_qiskit_transpiled)

     ┌──────────────┐                                                »
q_0: ┤ Gpi2(3.1416) ├────────────────────────────────────────────────»
     ├──────────────┤┌──────────────┐┌─────────────┐ ┌──────────────┐»
q_1: ┤ Gpi(0.94904) ├┤ Gpi2(3.3952) ├┤ Gpi(4.5054) ├─┤ Gpi2(4.0448) ├»
     ├─────────────┬┘├──────────────┤├─────────────┴┐└──────────────┘»
q_2: ┤ Gpi(3.4907) ├─┤ Gpi2(1.0618) ├┤ Gpi2(1.0625) ├────────────────»
     ├─────────────┤ ├──────────────┤├──────────────┤┌─────────────┐ »
q_3: ┤ Gpi(5.1226) ├─┤ Gpi2(4.9313) ├┤ Gpi2(4.4139) ├┤ Gpi(4.2225) ├─»
     └─────────────┘ └──────────────┘└──────────────┘└─────────────┘ »
«                                                                       »
«q_0: ──────────────────────────────────────────────────────────────────»
«     ┌────────────────────────────────┐┌─────────────┐ ┌──────────────┐»
«q_1: ┤0                               ├┤ Gpi2(5.315) ├─┤ Gpi2(1.3813) ├»
«     │  Partialms(0.90308,1.4259,π/2) │├─────────────┤ ├────────

# Convert Qiskit circuit to PennyLane


In [29]:
qml_circuit = qml.from_qiskit(qc_qiskit_transpiled)
print("Transpiled result",np.abs(qml.matrix(qml_circuit, wire_order=list(range(qc_qiskit.num_qubits)))().T[0])**2)#check
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

Transpiled result [0.1781696  0.00496901 0.06200014 0.00172914 0.00489825 0.17940388
 0.00170451 0.06242965 0.18044568 0.00503249 0.06427799 0.00179266
 0.00496083 0.18169574 0.00176714 0.06472328]
Not transpiled result: [0.17558095 0.18633726 0.00484553 0.00514237 0.05694874 0.06770506
 0.00157162 0.00186846 0.00484553 0.00514237 0.17558095 0.18633726
 0.00157162 0.00186846 0.05694874 0.06770506]


# NOISE

In [28]:
print("Noise 0     simulation result:", get_noisy_counts(qc_qiskit_transpiled,0.00,0.000,0.00))
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=list(range(qc_qiskit.num_qubits)))().T[0])**2)
print("Noisy     simulation result:", get_noisy_counts(qc_qiskit_transpiled,1000,0.01,0.001,0.01))#check

Noise 0     simulation result: [0.15527344 0.00488281 0.06152344 0.00097656 0.00585938 0.1953125
 0.         0.06347656 0.17382812 0.00585938 0.06835938 0.
 0.00878906 0.1875     0.00097656 0.06738281]
Noisy     simulation result: {'0000': 161, '0001': 14, '0010': 63, '0011': 4, '0100': 9, '0101': 164, '0110': 3, '0111': 62, '1000': 172, '1001': 11, '1010': 78, '1011': 4, '1100': 16, '1101': 166, '1110': 3, '1111': 70}
Noiseless simulation result: [0.1781696  0.00496901 0.06200014 0.00172914 0.00489825 0.17940388
 0.00170451 0.06242965 0.18044568 0.00503249 0.06427799 0.00179266
 0.00496083 0.18169574 0.00176714 0.06472328]


# Vibratium

In [22]:
bracket_qc=from_qasm2_to_braket("circuit.qasm")


In [23]:
print(bracket_qc)

T  : │        0        │      1      │      2       │           3            │      4       │      5       │      6      │           7            │      8       │      9       │      10      │      11       │ 12  │
                        ┌───────────┐ ┌────────────┐ ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌───────────┐ ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌────────────┐                 ┌───┐ 
q0 : ───StartVerbatim───┤ GPi(3.93) ├─┤ GPi2(3.14) ├─┤ MS(1.16, 3.06, 1.32) ├─┤ GPi2(0.64) ├─┤ GPi2(6.20) ├─┤ GPi(3.12) ├─┤ MS(1.61, 0.10, 1.37) ├─┤ GPi2(1.19) ├─┤ GPi2(1.24) ├─┤ GPi2(5.58) ├───EndVerbatim───┤ M ├─
              ║         └───────────┘ └────────────┘ └──────────┬───────────┘ └────────────┘ └────────────┘ └───────────┘ └──────────┬───────────┘ └────────────┘ └────────────┘ └────────────┘        ║        └───┘ 
              ║         ┌───────────┐                ┌──────────┴───────────┐ ┌────────────┐                              ┌──────────┴──────

In [24]:
device = LocalSimulator() # Choose the local simulator
shots = 100000  # Number of times to execute the circuit

result = device.run(bracket_qc, shots=shots).result()
counts = result.measurement_probabilities  # Get the shot results

print(counts)  # Dictionary {bitstring: counts}
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

{'11': 0.35773, '01': 0.123, '10': 0.41512, '00': 0.10415}
Noiseless simulation result: [0.10533638 0.12272559 0.41539821 0.35653982]


# GATE braket TEST

In [17]:
print(braket_Circuit().gpi(0,1))

T  : │    0     │
      ┌────────┐ 
q0 : ─┤ GPi(1) ├─
      └────────┘ 
T  : │    0     │


In [18]:
braket_Circuit().gpi(0,0.445).to_unitary()

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [19]:
phi=0.445
np.array([
            [0, np.exp(-1j * phi)],
            [np.exp(1j * phi), 0]
        ])

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [20]:
braket_Circuit().gpi2(0,0.445).to_unitary()

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [13]:
 # Extract the phase parameter
factor = 1 / np.sqrt(2)  # Normalization factor
exp_pos = -1j * np.exp(1j * phi)
exp_neg = -1j * np.exp(-1j * phi)

factor * np.array([
            [1, exp_neg],
            [exp_pos, 1]
        ])

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [14]:
phi1=0.2455
phi2=0.4157
theta=0.11545
q1=0
q2=1
braket_Circuit().ms(q1, q2, phi1, phi2,theta).to_unitary()

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [15]:
phi0=phi1
phi1=phi2

cos_theta = np.cos(theta/2)
sin_theta = np.sin(theta/2)

e_pos = -1j * np.exp(1j * (phi0 + phi1))
e_neg = -1j * np.exp(-1j * (phi0 + phi1))
e_diff_pos = -1j * np.exp(1j * (phi0 - phi1))
e_diff_neg = -1j * np.exp(-1j * (phi0 - phi1))

np.array([
    [cos_theta, 0, 0, e_neg * sin_theta],
    [0, cos_theta, e_diff_neg * sin_theta, 0],
    [0, e_diff_pos * sin_theta, cos_theta, 0],
    [e_pos * sin_theta, 0, 0, cos_theta]
])

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [16]:
braket_Circuit().zz(0,1,np.pi).to_unitary()

array([[6.123234e-17-1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17-1.j]])